In [106]:
import numpy as np
import pandas as pd
import random
import sqlite3

In [107]:
# Create the connection to the sqlite db
database = 'db.db'
conn = sqlite3.connect(database)

### Page Statistics Global

Now we are onto the actual sql part.

#### How many new subscribers for the page over the time period?
Logic is sum the NumberofFans at the oldest date from the FansPerCountry table and then subtract that from the sum of NumberofFans at the earliest date. There are NumberofFans per Country that is why you have to sum and then subtract.

In [108]:
# Get last date in table
sql = 'SELECT Date FROM FansPerCountry ORDER BY Date DESC LIMIT 1'
last_date = pd.read_sql(sql, conn)
last_date = last_date['Date'][0]
last_date

'2018-10-16'

In [109]:
# Get first date in table
sql = 'SELECT Date FROM FansPerCountry ORDER BY Date LIMIT 1'
first_date = pd.read_sql(sql, conn)
first_date = first_date['Date'][0]
first_date

'2018-09-19'

In [110]:
# total_last_day
sql = """SELECT SUM(NumberOfFans)
            FROM FansPerCountry
                WHERE Date = '{}'""".format(last_date)
total_last_day = pd.read_sql(sql, conn)

# total_first_day
sql = """SELECT SUM(NumberOfFans)
            FROM FansPerCountry
                WHERE Date = '{}'""".format(first_date)
total_first_day = pd.read_sql(sql, conn)

# increase calculation
increase = total_last_day - total_first_day
print('Increase in NumberOfFans over the period is', increase.iloc[0,0])

Increase in NumberOfFans over the period is 41571


In [111]:
# Solving it in one SQL query
sql = """
SELECT (SUM(NumberOfFans) - NumberOfFansStart) AS IncreasedFans
    FROM FansPerCountry, 
    (SELECT sum(NumberOfFans) as NumberOfFansStart 
        FROM FansPerCountry 
            WHERE Date = (Select MIN(Date) FROM GlobalPage)) AS From_Table
    WHERE Date = (Select MAX(Date) FROM GlobalPage)
"""
df = pd.read_sql(sql, conn)
df

,IncreasedFans
0,41571


#### What is the daily average reach of the posts on the page over the period?
Logic is sum up all of the posts in DailyPostsReach from the GlobalPage and take the average.

In [112]:
sql = """
SELECT Date, SUM(DailyPostsReach) as DailySum
    FROM GlobalPage
        GROUP BY DATE"""
daily_sum = pd.read_sql(sql, conn)
print('daily_sum shape is', daily_sum.shape)
daily_sum.head(5)

daily_sum shape is (28, 2)


,Date,DailySum
0,2018-09-19,82110362
1,2018-09-20,82247928
2,2018-09-21,82378374
3,2018-09-22,82495670
4,2018-09-23,82614344


In [113]:
print('Average number of DailyPostsReach is', daily_sum['DailySum'].mean().astype(int))

Average number of DailyPostsReach is 83826721


In [114]:
# or more correct just using SQL
sql = """
SELECT AVG(DailySum)
    FROM
        (SELECT Date, SUM(DailyPostsReach) as DailySum
             FROM GlobalPage
                 GROUP BY DATE) as inner_query"""
df = pd.read_sql(sql, conn)
df

,AVG(DailySum)
0,8.382672e+07


#### Daily average engagement (EngagedFans) using PostInsights over the period?
Logic is take the SUM of the CreatedTime by Day and then take the average of EngagedFans

In [115]:
print('Average number of daily EngagedFansSum is', df['AVG(DailySum)'].mean().astype(int))

Average number of daily EngagedFansSum is 83826721


In [116]:
# or more correct just using SQL
sql = """
SELECT AVG(EngagedFansSum) as DailyAvgEngagementRate
    FROM
        (SELECT CreatedTime, SUM(EngagedFans) AS EngagedFansSum
            FROM PostInsights
                GROUP BY strftime('%Y-%m-%d', `CreatedTime`)) as inner_query"""
df = pd.read_sql(sql, conn)
df

,DailyAvgEngagementRate
0,61128.071429


In [117]:
# Daily average NewLikes rate on the page over the period? 
# Queston 3 on the first SQL slide
sql = """
Select ROUND(AVG(likes)) 
            FROM (select AVG(NewLikes) AS likes 
                FROM GlobalPage 
                    GROUP BY Date);
"""
df = pd.read_sql(sql, conn)
df

,ROUND(AVG(likes))
0,8943.0


In [118]:
# Lets prove in in Pandas
sql = """
SELECT Date, NewLikes 
    FROM GlobalPage
"""
df = pd.read_sql(sql, conn)
df.head()

,Date,NewLikes
0,2018-09-19,2458
1,2018-09-19,4319
2,2018-09-19,18130
3,2018-09-19,1330
4,2018-09-19,5660


In [119]:
NewLikes_By_Date = df.groupby('Date').mean()
NewLikes_By_Date.head()

,NewLikes
Date,
2018-09-19,8759.466667
2018-09-20,8774.111111
2018-09-21,8787.977778
2018-09-22,8800.533333
2018-09-23,8813.222222


In [120]:
NewLikes_By_Date['NewLikes'].mean()

8942.556349206348

#### What are the top 10 countries (considering the number of fans)?

In [121]:
sql = """
SELECT 
    PopStats.CountryName, 
    PopStats.CountryCode, 
    FansPerCountry.NumberOfFans
FROM PopStats 
    JOIN FansPerCountry ON PopStats.CountryCode = FansPerCountry.CountryCode
        WHERE FansPerCountry.Date = '{}'
            ORDER BY FansPerCountry.NumberOfFans DESC
                LIMIT 10""".format(last_date)
top_10 = pd.read_sql(sql, conn)
top_10

,CountryName,CountryCode,NumberOfFans
0,Ivory Coast,CI,112160
1,Cameroon,CM,102211
2,Senegal,SN,83561
3,France,FR,73252
4,Madagascar,MG,72956
5,Democratic Republic of the Congo,CD,50705
6,Burkino Faso,BF,43500
7,Mali,ML,40578
8,Algeria,DZ,39093
9,Guinea,GN,36821


#### What are the top 10 countries (considering the penetration ratio: % of the country population that are fans)?
Logic is NumerOfFans/Population of that country Limit 10

In [122]:
sql = """
SELECT 
    PopStats.CountryName, 
    PopStats.CountryCode, 
    FansPerCountry.NumberOfFans,
    PopStats.Population,
    100.0 * FansPerCountry.NumberOfFans / PopStats.Population AS Percent_Penetration
FROM FansPerCountry
    JOIN PopStats ON PopStats.CountryCode = FansPerCountry.CountryCode
        WHERE FansPerCountry.Date = '{}'
            ORDER BY Percent_Penetration DESC
                LIMIT 10""".format(last_date)

top_10 = pd.read_sql(sql, conn)
top_10

,CountryName,CountryCode,NumberOfFans,Population,Percent_Penetration
0,Reunion,RE,20885,866506,2.410255
1,French Polynesia,PF,5148,283007,1.819036
2,New Caledonia,NC,5032,280460,1.794195
3,Mauritius,MU,24210,1364283,1.774559
4,Martinique,MQ,5427,376480,1.441511
5,Guadeloupe,GP,5379,395700,1.359363
6,Gabon,GA,23954,2119036,1.130420
7,Mayotte,YT,1983,270372,0.733434
8,Comoros,KM,4925,821164,0.599758
9,French Guiana,GF,1687,296711,0.568567


In [123]:
# or
sql = """
SELECT PopStats.CountryName,
    substr(CAST (FansPerCountry.NumberOfFans*1.0/PopStats.Population *100 as string),1,4) || '%'  as Penetration_Ratio
        FROM FansPerCountry, PopStats
        WHERE 
            PopStats.CountryCode = FansPerCountry.CountryCode
            AND Date = (SELECT MAX(Date) FROM FansPerCountry)
                ORDER BY Penetration_Ratio DESC
                    LIMIT 10
"""
top_10 = pd.read_sql(sql, conn)
top_10

,CountryName,Penetration_Ratio
0,Reunion,2.41%
1,French Polynesia,1.81%
2,New Caledonia,1.79%
3,Mauritius,1.77%
4,Martinique,1.44%
5,Guadeloupe,1.35%
6,Gabon,1.13%
7,Mayotte,0.73%
8,Comoros,0.59%
9,French Guiana,0.56%


#### What are the bottom 10 cities (considering the number of fans) among countries with a population over 20 million? This could be considered our growth potential

Logic is select just the countries with over 20 million limit 10 

In [124]:
sql = """
SELECT 
    p.CountryName, 
    p.CountryCode, 
    f.City,
    f.NumberOfFans,
    p.Population
FROM FansPerCity AS f
    JOIN PopStats AS p 
        ON p.CountryCode = f.CountryCode
            WHERE p.Population > 20000000 AND f.Date = (SELECT MAX(Date) FROM FansPerCity)
                GROUP BY f.City
                    ORDER BY f.NumberOfFans
                        LIMIT 10"""

bottom_10 = pd.read_sql(sql, conn)
bottom_10

,CountryName,CountryCode,City,NumberOfFans,Population
0,Algeria,DZ,Bejaia,2301,41657488
1,Cameroon,CM,Ngaoundere,2318,25640965
2,Madagascar,MG,Fianarantsoa,2366,25683610
3,Algeria,DZ,Tizi Ouzou,2524,41657488
4,Canada,CA,Montreal,2887,35881659
5,Algeria,DZ,Oran,2920,41657488
6,Ivory Coast,CI,Bouake,3376,24290000
7,Ivory Coast,CI,Cocody,3647,24290000
8,Morocco,MA,Casablanca,3951,34314130
9,Angola,AO,Luanda,4614,30355880


#### What is the split of page fans across age groups (in %)?

Logic is group by AgeGroup and summed

In [125]:
# Get Total NumbeOfFans from FansPerGenderAge
sql = """
SELECT 
    SUM(FansPerGenderAge.NumberOfFans) as Total_NumberOfFans
        FROM FansPerGenderAge
            WHERE FansPerGenderAge.Date = '{}' """.format(last_date)

total_fans = pd.read_sql(sql, conn)
total_fans = total_fans.iloc[0,0]
print('The total NumberOfFans in the FansPerGenderAge table is', total_fans)

The total NumberOfFans in the FansPerGenderAge table is 971011


In [126]:
sql = """
SELECT 
    AgeGroup, 
    SUM(NumberOfFans) AS Fans,
    {} AS Total,
    100.0 * SUM(NumberOfFans) / {} AS Percentage
        FROM FansPerGenderAge
            WHERE Date = '{}'
                GROUP BY AgeGroup""".format(total_fans, total_fans, last_date)

Age_Group_NumberOfFans = pd.read_sql(sql, conn)
Age_Group_NumberOfFans

,AgeGroup,Fans,Total,Percentage
0,13-17,20332,971011,2.093900
1,18-24,206787,971011,21.296051
2,25-34,347581,971011,35.795784
3,35-44,188335,971011,19.395764
4,45-54,91804,971011,9.454476
5,55-64,48793,971011,5.024969
6,65+,67379,971011,6.939056


In [127]:
# or
sql = """
SELECT AgeGroup, 
       SUM(NumberOfFans),
       SUM(NumberOfFans) * 1.0 /
           (SELECT sum(NumberOfFans) 
               FROM FansPerGenderAge
                   WHERE Date = (SELECT MAX(Date) FROM FansPerGenderAge)) * 100.0 AS NumberOfFans_Percentage 
            FROM FansPerGenderAge
                WHERE Date = (SELECT MAX(Date) FROM FansPerGenderAge)
                    GROUP BY AgeGroup
"""
Age_Group_NumberOfFans = pd.read_sql(sql, conn)
Age_Group_NumberOfFans

,AgeGroup,SUM(NumberOfFans),NumberOfFans_Percentage
0,13-17,20332,2.093900
1,18-24,206787,21.296051
2,25-34,347581,35.795784
3,35-44,188335,19.395764
4,45-54,91804,9.454476
5,55-64,48793,5.024969
6,65+,67379,6.939056


#### What is the split of page fans by gender (in %)?

Logic is group by gender

In [128]:
sql = """
SELECT 
    Gender, 
    SUM(NumberOfFans) AS Fans,
    {} AS Total,
    100.0 * SUM(NumberOfFans) / {} AS Percentage
        FROM FansPerGenderAge
            WHERE Date = '{}'
                GROUP BY Gender""".format(total_fans, total_fans, last_date)

Age_Group_NumberOfFans = pd.read_sql(sql, conn)
Age_Group_NumberOfFans

,Gender,Fans,Total,Percentage
0,F,547728,971011,56.408012
1,M,422365,971011,43.497448
2,U,918,971011,0.094541


In [129]:
# or
sql = """
SELECT Gender, 
    SUM(NumberOfFans),
    SUM(NumberOfFans) * 1.0 /
        (SELECT sum(NumberOfFans) 
            FROM FansPerGenderAge
                WHERE Date = (SELECT MAX(Date) FROM FansPerGenderAge)) * 100 AS NumberOfFans_Percentage 
        FROM FansPerGenderAge
            WHERE Date = (SELECT MAX(Date) FROM FansPerGenderAge)
                GROUP BY Gender
"""
Age_Group_NumberOfFans = pd.read_sql(sql, conn)
Age_Group_NumberOfFans

,Gender,SUM(NumberOfFans),NumberOfFans_Percentage
0,F,547728,56.408012
1,M,422365,43.497448
2,U,918,0.094541


#### What is the number and percentage of the page fans that have declared English as their primary language (in %)?

In [130]:
# Get Total NumbeOfFans from FansPerLanguage
sql = """
SELECT 
    SUM(NumberOfFans) as Total_NumberOfFans
        FROM FansPerLanguage
            WHERE Language = 'en' AND 
                  Date = '{}' """.format(last_date)

total_fans = pd.read_sql(sql, conn)
total_fans = total_fans.iloc[0,0]
print('The total NumberOfFans in the FansPerLanguage table is', total_fans)

The total NumberOfFans in the FansPerLanguage table is 49418


In [131]:
sql = """
SELECT 
    CountryCode,
    Language, 
    SUM(NumberOfFans) AS Fans,
    {} AS Total,
    100.0 * SUM(NumberOfFans) / {} AS Percentage
        FROM FansPerLanguage
            WHERE Date = '{}' AND Language = 'en'
                GROUP BY CountryCode""".format(total_fans, total_fans, last_date)

Language_NumberOfFans = pd.read_sql(sql, conn)
Language_NumberOfFans

,CountryCode,Language,Fans,Total,Percentage
0,GB,en,10509,49418,21.265531
1,US,en,38909,49418,78.734469


In [132]:
total_english_speakers = Language_NumberOfFans.sum()
total_english_speakers

CountryCode     GBUS
Language        enen
Fans           49418
Total          98836
Percentage       100
dtype: object

In [133]:
# or
sql = """
SELECT ROUND( 
        (SELECT sum(NumberOfFans) 
              FROM FansPerLanguage 
                  WHERE Date = (SELECT MAX(Date) FROM FansPerLanguage) AND 
                      Language = 'en' GROUP BY Language) * 100.0 / 
                          SUM(NumberOfFans),4) AS PercentageEnglighSpeakingFans
            FROM FansPerLanguage
                WHERE Date = (SELECT MAX(Date) FROM FansPerLanguage)
"""

Language_NumberOfFans = pd.read_sql(sql, conn)
Language_NumberOfFans

,PercentageEnglighSpeakingFans
0,5.0907


#### Based on the number of fans who have declared English as their primary language and living in the US, what is the potential buying power that can be accessed? 

(Please use the average income data per country for this question. You can use the latest year available. It is estimated that on average, 0.01% of the annual income is dedicated to online magazine subscriptions in US)

This is an EXTREMELY low number of fans for the USA. These are fans of the company so it could be low. 

In [134]:
sql = """
SELECT 
    PopStats.CountryCode,
    PopStats.CountryName,
    FansPerLanguage.Language, 
    FansPerLanguage.NumberOfFans, 
    PopStats.Population,
    PopStats.AverageIncome,
    (.0001 * FansPerLanguage.NumberofFans * PopStats.AverageIncome) AS Buying_Power
FROM FansPerLanguage
    JOIN PopStats on PopStats.CountryCode = FansPerLanguage.CountryCode
        WHERE FansPerLanguage.Date = '{}' AND 
            PopStats.CountryCode ='US'""".format(last_date)

Buying_Power = pd.read_sql(sql, conn)
print('The Buying_Power available is', Buying_Power.iloc[0,-1].round(2))
Buying_Power.head()

The Buying_Power available is 200322.75


,CountryCode,CountryName,Language,NumberOfFans,Population,AverageIncome,Buying_Power
0,US,United states,en,38909,329256465,51484.94,200322.753046


#### What is the split of posts engagement ratio per day of the week (monday, tuesday,...)?

In [135]:
# Get the total of all EngagedFans
sql = """
SELECT 
    strftime('%w',PostInsights.CreatedTime) AS Day,
    SUM(EngagedFans) AS TotalEngagedFans
FROM PostInsights 
    GROUP BY strftime('%w', PostInsights.CreatedTime)"""

Day_Of_Week = pd.read_sql(sql, conn)
Day_Of_Week = Day_Of_Week.sum()
TotalEngagedFans = Day_Of_Week['TotalEngagedFans']
print('TotalEngagedFans are', TotalEngagedFans)

TotalEngagedFans are 1711586.0


In [136]:
# Get the total of all EngagedFans
sql = """
SELECT 
    strftime('%w',PostInsights.CreatedTime) AS Day,
    SUM(EngagedFans) AS TotalEngagedFans
FROM PostInsights 
    GROUP BY strftime('%w', PostInsights.CreatedTime)"""

df = pd.read_sql(sql, conn)
TotalEngagedFans = df['TotalEngagedFans'].sum()
print('TotalEngagedFans are', TotalEngagedFans)

TotalEngagedFans are 1711586


In [137]:
sql = """
SELECT 
    strftime('%w', PostInsights.CreatedTime) AS Day,
    SUM(EngagedFans) AS EngagedFans,
    (100.0 * SUM(EngagedFans) / {}) AS EngagedFansPercent
FROM PostInsights 
    GROUP BY strftime('%w', PostInsights.CreatedTime)
        ORDER BY EngagedFansPercent DESC""".format(TotalEngagedFans)

Day_Of_Week = pd.read_sql(sql, conn)
Day_Of_Week

,Day,EngagedFans,EngagedFansPercent
0,6,337621,19.725623
1,1,329207,19.234032
2,2,319550,18.669819
3,3,263253,15.380647
4,0,206805,12.082653
5,5,146943,8.585195
6,4,108207,6.322031


#### What is the best day of the week to publish posts?

In [138]:
# Solution that does not require calculating total first
sql = """
SELECT 
    strftime('%w', PostInsights.CreatedTime) AS WeekDay,  
    substr(SUM(EngagedFans) * 1.0 /
        (SELECT sum(EngagedFans) 
            FROM PostInsights) * 100, 1, 5) || " %" AS Engagement_Ratio
    FROM PostInsights
        GROUP BY WeekDay
"""
Day_Of_Week = pd.read_sql(sql, conn)
Day_Of_Week

,WeekDay,Engagement_Ratio
0,0,12.08 %
1,1,19.23 %
2,2,18.66 %
3,3,15.38 %
4,4,6.322 %
5,5,8.585 %
6,6,19.72 %


In [139]:
Day_Of_Week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
WeekDay             7 non-null object
Engagement_Ratio    7 non-null object
dtypes: object(2)
memory usage: 240.0+ bytes


In [140]:
# Add day of week as a 'word'
days = {'0':'Sunday', '1':'Monday', '2':'Tuesday', '3':'Wednesday', '4':'Thursday', '5':'Friday', '6':'Saturday'}
Day_Of_Week['Day'] = Day_Of_Week['WeekDay'].apply(lambda x: days[x])
Day_Of_Week.set_index('WeekDay', drop=True, inplace=True)
Day_Of_Week = Day_Of_Week[['Day', 'Engagement_Ratio']]
Day_Of_Week

,Day,Engagement_Ratio
WeekDay,,
0,Sunday,12.08 %
1,Monday,19.23 %
2,Tuesday,18.66 %
3,Wednesday,15.38 %
4,Thursday,6.322 %
5,Friday,8.585 %
6,Saturday,19.72 %


In [141]:
# Best solution
sql = """
SELECT ROUND(SUM(EngagedFans)*100./(SELECT SUM(EngagedFans) 
FROM PostInsights),2) as EngagedFansPercent,
    CASE CAST (strftime('%w', createdtime) AS INTEGER)
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        ELSE 'Saturday' END AS Day
FROM PostInsights
    GROUP BY DAY
        ORDER BY EngagedFansPercent DESC"""

Day_Of_Week = pd.read_sql(sql, conn)
Day_Of_Week

,EngagedFansPercent,Day
0,19.73,Saturday
1,19.23,Monday
2,18.67,Tuesday
3,15.38,Wednesday
4,12.08,Sunday
5,8.59,Friday
6,6.32,Thursday


#### What are the split of posts engagement ratio per time of the day ?

In [142]:
sql = """
SELECT 
    strftime('%H',PostInsights.CreatedTime) AS Hour,
    SUM(EngagedFans) AS SumEngagedFans,
    (100.0 * SUM(EngagedFans) / {}) AS EngagedFansPercent
FROM PostInsights 
    GROUP BY strftime('%H', PostInsights.CreatedTime)""".format(TotalEngagedFans)

Hour_Of_The_Day = pd.read_sql(sql, conn)
Hour_Of_The_Day

,Hour,SumEngagedFans,EngagedFansPercent
0,05,279770,16.345658
1,06,110964,6.483110
2,07,96864,5.659312
3,08,93678,5.473169
4,09,93047,5.436303
5,10,70956,4.145629
6,11,98483,5.753903
7,12,82313,4.809165
8,13,67949,3.969944
9,14,67083,3.919347


In [143]:
Hour_Of_The_Day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 3 columns):
Hour                  19 non-null object
SumEngagedFans        19 non-null int64
EngagedFansPercent    19 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 584.0+ bytes


In [144]:
TimeOfDayRangeDict = {'05': '05:00 - 08:59', 
                      '06': '05:00 - 08:59', 
                      '07': '05:00 - 08:59',
                      '08': '05:00 - 08:59',
                      '09': '09:00 - 11:59',
                      '10': '09:00 - 11:59',
                      '11': '09:00 - 11:59',
                      '12': '12:00 - 14:59',
                      '13': '12:00 - 14:59',
                      '14': '12:00 - 14:59',
                      '15': '15:00 - 18:59',
                      '16': '15:00 - 18:59',
                      '17': '15:00 - 18:59',
                      '18': '15:00 - 18:59',
                      '19': '19:00 - 21:59',
                      '20': '19:00 - 21:59',
                      '21': '19:00 - 21:59',
                      '22': '22:00 or later',
                      '23': '22:00 or later'}   

In [145]:
Hour_Of_The_Day['TimeOfDayRange'] = Hour_Of_The_Day['Hour'].apply(lambda x: TimeOfDayRangeDict[x])
Hour_Of_The_Day.tail(5)

,Hour,SumEngagedFans,EngagedFansPercent,TimeOfDayRange
14,19,83411,4.873316,19:00 - 21:59
15,20,72668,4.245653,19:00 - 21:59
16,21,68507,4.002545,19:00 - 21:59
17,22,83884,4.900952,22:00 or later
18,23,44122,2.577843,22:00 or later


#### What is the best time of the day to publish posts?

In [146]:
hour_gb = Hour_Of_The_Day.groupby('TimeOfDayRange').sum()
print('The best time of day to post is', hour_gb.index[0])
hour_gb['EngagedFansPercent'] = hour_gb['EngagedFansPercent'].round(2)
hour_gb.sort_values('EngagedFansPercent', ascending=False)

The best time of day to post is 05:00 - 08:59


,SumEngagedFans,EngagedFansPercent
TimeOfDayRange,,
05:00 - 08:59,581276,33.96
15:00 - 18:59,297887,17.40
09:00 - 11:59,262486,15.34
19:00 - 21:59,224586,13.12
12:00 - 14:59,217345,12.70
22:00 or later,128006,7.48


In [147]:
# Now solve it using SQL and a table

In [148]:
# Have created a csv with DayHour and TimeOfDayRange, using 24 hours!!
# SQLite uses strings for dates. Make sure Day/Hour is a string
df = pd.read_csv(r'TimeDay.csv', dtype={'DayHour': object})
df.head()

,DayHour,TimeOfDayRange
0,01,00:00 - 04:59
1,02,00:00 - 04:59
2,03,00:00 - 04:59
3,04,00:00 - 04:59
4,05,05:00 - 08:59


In [149]:
# Create TimeDay table
df.to_sql('TimeDay', conn, if_exists="replace", index=False)

# Read in and make sure it is what I expect
df = pd.read_sql('SELECT * FROM TimeDay', conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 2 columns):
DayHour           23 non-null object
TimeOfDayRange    23 non-null object
dtypes: object(2)
memory usage: 496.0+ bytes


In [150]:
df.head()

,DayHour,TimeOfDayRange
0,01,00:00 - 04:59
1,02,00:00 - 04:59
2,03,00:00 - 04:59
3,04,00:00 - 04:59
4,05,05:00 - 08:59


In [151]:
# Final
sql = """SELECT t.TimeOfDayRange,
                SUM(p.EngagedFans) AS SumEngagedFans,
                ROUND((100.0 * SUM(EngagedFans) / {}),2) AS EngagedFansPercent
         FROM PostInsights as p
             JOIN TimeDay as t
                 ON STRFTIME('%H', p.CreatedTime) = t.DayHour
                     GROUP BY t.TimeOfDayRange
                         ORDER BY EngagedFansPercent DESC""".format(TotalEngagedFans)
df = pd.read_sql(sql, conn)
df

,TimeOfDayRange,SumEngagedFans,EngagedFansPercent
0,05:00 - 08:59,581276,33.96
1,15:00 - 18:59,297887,17.40
2,09:00 - 11:59,262486,15.34
3,19:00 - 21:59,224586,13.12
4,12:00 - 14:59,217345,12.70
5,22:00 or later,128006,7.48


In [152]:
# Or
sql = """
SELECT 
    t.TimeOfDayRange, 
    substr(sum(EngagedFans) * 1.0/ (SELECT sum(EngagedFans) FROM PostInsights) * 100,1,5) || '%' AS Engagement_Ratio
        FROM PostInsights as p
            JOIN TimeDay as t
                ON STRFTIME('%H', p.CreatedTime) = t.DayHour
                    GROUP BY t.TimeOfDayRange"""
df = pd.read_sql(sql, conn)
df

,TimeOfDayRange,Engagement_Ratio
0,05:00 - 08:59,33.96%
1,09:00 - 11:59,15.33%
2,12:00 - 14:59,12.69%
3,15:00 - 18:59,17.40%
4,19:00 - 21:59,13.12%
5,22:00 or later,7.478%


In [153]:
# Or (don't use this one)
sql = """
SELECT ROUND(sum(EngagedFans)*1.0/(SELECT SUM(EngagedFans) 
    FROM PostInsights)*100.0,2)||'%'Percentage, 
        case when cast(strftime('%H',CreatedTime) as integer) >= 5 and 
        cast(strftime('%H',CreatedTime) as integer) <= 8 then 5 when 
        cast(strftime('%H',CreatedTime) as integer) >= 9 and 
        cast(strftime('%H',CreatedTime) as integer) <= 11 then 9 when 
        cast(strftime('%H',CreatedTime) as integer) >= 12 and 
        cast(strftime('%H',CreatedTime) as integer) <= 14 then 12 when 
        cast(strftime('%H',CreatedTime) as integer) >= 15 and 
        cast(strftime('%H',CreatedTime) as integer) <= 18 then 15 when 
        cast(strftime('%H',CreatedTime) as integer) >= 19 and 
        cast(strftime('%H',CreatedTime) as integer) <= 21 then 19 else 22 
            end as timecat 
                from PostInsights 
                    group by timecat;
"""
df = pd.read_sql(sql, conn)
df

,Percentage,timecat
0,33.96%,5
1,15.34%,9
2,12.7%,12
3,17.4%,15
4,13.12%,19
5,7.48%,22


| CountryName | City | NumberOfFans | Population |
| :--- | :--- | :--- | :--- |
| Algeria | Bejaia | 2301 | 41657488 |
| Cameroon | Ngaoundere | 2318 | 25640965 |
| Madagascar | Fianarantsoa | 2366 | 25683610 |
| Algeria | Tizi Ouzou | 2524 | 41657488 |
| Canada | Montreal | 2887 | 35881659 |
| Algeria | Oran | 2920 | 41657488 |
| Ivory Coast | Bouake  | 3376 | 24290000 |
| Ivory Coast | Cocody | 3647 | 24290000 |
| Morocco | Casablanca | 3951 | 34314130 |
| Angola | Luanda | 4614 | 30355880 |

| AgeGroup | Percentage |
| :--- | :--- |
| 13-17 | 2.09% |
| 18-24 | 21.3% |
| 25-34 | 35.8% |
| 35-44 | 19.4% |
| 45-54 | 9.45% |
| 55-64 | 5.02% |
| 65+ | 6.94% |